## Testing your algorithm on your local machine

When you're packaging you first algorithm to use with Amazon SageMaker, you probably want to test it yourself to make sure it's working correctly. We use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to test both locally and on SageMaker. For more examples with the SageMaker Python SDK, see [Amazon SageMaker Examples](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk). In order to test our algorithm, we need our dataset.

## SageMaker Python SDK Local Training
To represent our training, we use the Estimator class, which needs to be configured in five steps. 
1. IAM role - our AWS execution role
2. train_instance_count - number of instances to use for training.
3. train_instance_type - type of instance to use for training. For training locally, we specify `local`.
4. image_name - our custom TensorFlow Docker image we created.
5. hyperparameters - hyperparameters we want to pass.

Let's start with setting up our IAM role. We make use of a helper function within the Python SDK. This function throw an exception if run outside of a SageMaker notebook instance, as it gets metadata from the notebook instance. If running outside, you must provide an IAM role with proper access stated above in [Permissions](#Permissions).

In [1]:
from sagemaker import get_execution_role

role = get_execution_role()

## Fit, Deploy, Predict

Now that the rest of our estimator is configured, we can call `fit()` with the path to our local CIFAR10 dataset prefixed with `file://`. This invokes our TensorFlow container with 'train' and passes in our hyperparameters and other metadata as json files in /opt/ml/input/config within the container.

After our training has succeeded, our training algorithm outputs our trained model within the /opt/ml/model directory, which is used to handle predictions.

We can then call `deploy()` with an instance_count and instance_type, which is 1 and `local`. This invokes our Tensorflow container with 'serve', which setups our container to handle prediction requests through TensorFlow Serving. What is returned is a predictor, which is used to make inferences against our trained model.

After our prediction, we can delete our endpoint.

We recommend testing and training your training algorithm locally first, as it provides quicker iterations and better debuggability.

In [2]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [4]:
from sagemaker.estimator import Estimator
import subprocess

hyperparameters = {'algo-name': 'DeepAR', 'freq': '1H', 'prediction-length': 3*24, 'context-length': 7*24, 'epochs': 2}

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_uri='gluonts_sagemaker:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///home/ec2-user/SageMaker/gluonts_sagemaker/data/')

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Instance type = local
Creating nwrcsnrblm-algo-1-uio4l ... 
Creating nwrcsnrblm-algo-1-uio4l ... done
Attaching to nwrcsnrblm-algo-1-uio4l
nwrcsnrblm-algo-1-uio4l | train_cmd: ['/usr/local/bin/python', 'main.py', '--model-dir', '/opt/ml/model', '--algo-name', 'DeepAR', '--freq', '1H', '--prediction-length', '72', '--context-length', '168', '--epochs', '2']
nwrcsnrblm-algo-1-uio4l | Training complete.
nwrcsnrblm-algo-1-uio4l exited with code 0
Aborting on container exit...
===== Job Complete =====


In [ ]:
# predictor = estimator.deploy(1, instance_type)

## Making predictions using Python SDK

To make predictions, we use an image that is converted using OpenCV into a json format to send as an inference request. We need to install OpenCV to deserialize the image that is used to make predictions.

The JSON reponse will be the probabilities of the image belonging to one of the 10 classes along with the most likely class the picture belongs to. The classes can be referenced from the [CIFAR-10 website](https://www.cs.toronto.edu/~kriz/cifar.html). Since we didn't train the model for that long, we aren't expecting very accurate results.

In [ ]:
# from sagemaker.predictor import json_serializer, json_deserializer

# data = {'input_ids': [101, 2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#        'input_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#        'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#        'label_ids': [9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

# # The request and response format is JSON for TensorFlow Serving.
# # For more information: https://www.tensorflow.org/serving/api_rest#predict_api
# predictor.accept = 'application/json'
# predictor.content_type = 'application/json'

# predictor.serializer = json_serializer
# predictor.deserializer = json_deserializer

# # For more information on the predictor class.
# # https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
# predictor.predict(data)

In [ ]:
# predictor.delete_endpoint()

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker
Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment
Here we specify the bucket to use and the role that is used for working with SageMaker.

In [5]:
# S3 prefix
prefix = 'gluonts_sagemaker'

## Create the session

The session remembers our connection parameters to SageMaker. We use it to perform all of our SageMaker operations.

In [6]:
import sagemaker as sage

sess = sage.Session()

## Upload the data for training

We will use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [7]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/gluonts_sagemaker/data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
print(data_location)

s3://sagemaker-cn-northwest-1-343958593302/gluonts_sagemaker


## Training on SageMaker
Training a model on SageMaker with the Python SDK is done in a way that is similar to the way we trained it locally. This is done by changing our train_instance_type from `local` to one of our [supported EC2 instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/).

In addition, we must now specify the ECR image URL, which we just pushed above.

Finally, our local training dataset has to be in Amazon S3 and the S3 URL to our dataset is passed into the `fit()` call.

Let's first fetch our ECR image url that corresponds to the image we just built and pushed.

In [8]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'gluonts_sagemaker'

if region.startswith('cn'):
    ecr_image = '{}.dkr.ecr.{}.amazonaws.com.cn/{}:latest'.format(account, region, algorithm_name)
else:
    ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

343958593302.dkr.ecr.cn-northwest-1.amazonaws.com.cn/gluonts_sagemaker:latest


In [11]:
from sagemaker.estimator import Estimator

hyperparameters = {'algo-name': 'DeepAR', 'freq': '1H', 'prediction-length': 3*24, 'context-length': 7*24, 'epochs': 2}

instance_type = 'ml.c5.2xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
#                       image_name=ecr_image,
                      image_uri=ecr_image,
                      hyperparameters=hyperparameters,
                      base_job_name=algorithm_name.replace('_', '-'))

estimator.fit(data_location)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-04-01 03:45:34 Starting - Starting the training job...
2021-04-01 03:45:35 Starting - Launching requested ML instances......
2021-04-01 03:46:37 Starting - Preparing the instances for training...
2021-04-01 03:47:22 Downloading - Downloading input data...
2021-04-01 03:47:35 Training - Downloading the training image........train_cmd: ['/usr/local/bin/python', 'main.py', '--model-dir', '/opt/ml/model', '--algo-name', 'DeepAR', '--context-length', '168', '--freq', '1H', '--prediction-length', '72', '--epochs', '2']

2021-04-01 03:49:06 Training - Training image download completed. Training in progress.
2021-04-01 03:50:03 Uploading - Uploading generated training modelTraining complete.

2021-04-01 03:50:09 Completed - Training job completed
Training seconds: 167
Billable seconds: 167


In [ ]:
instance_type = 'ml.m5.xlarge'
predictor = estimator.deploy(1, instance_type)

-

In [ ]:
import re
import os
import pickle
import tokenization
import numpy as np

In [ ]:
def seg_char(sent):
    """
    把句子按字分开，不破坏英文结构
    """
    # 首先分割 英文 以及英文和标点
    pattern_char_1 = re.compile(r'([\W])')
    parts = pattern_char_1.split(sent)
    parts = [p for p in parts if len(p.strip())>0]
    # 分割中文
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = pattern.split(sent)
    chars = [w for w in chars if len(w.strip())>0]
    return chars

In [ ]:
max_seq_length = hyperparameters['max_seq_length']
vocab_file = hyperparameters['vocab_file']
print(max_seq_length)
print(vocab_file)

tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=True)

with open(os.path.join('albert_base_ner_checkpoints', 'label2id.pkl'), 'rb') as rf:  # TODO need use label2id.pkl in model.tar.gz
    label2id = pickle.load(rf)
    id2label = {value: key for key, value in label2id.items()}
print(label2id)
print(id2label)

In [ ]:
text = '因有关日寇在京掠夺文物详情，藏界较为重视，也是我们收藏北京史料中的要件之一。'
tokens = ['[CLS]']
tokens.extend(seg_char(text)[:max_seq_length-2])
tokens.append('[SEP]')
print(len(tokens), tokens)

input_ids = tokenizer.convert_tokens_to_ids(tokens)
for i in range(max_seq_length-len(tokens)):
    input_ids.append(0)

print(input_ids)

In [ ]:
from sagemaker.predictor import json_serializer, json_deserializer

data = {'instances': [{'input_ids': [input_ids]}]}

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

result = predictor.predict(data)
print(result)

In [ ]:
result_label = [id2label[id] for id in result['predictions'][0] if id != 0]
print(result_label)

## Optional cleanup
When you're done with the endpoint, you should clean it up.

All of the training jobs, models and endpoints we created can be viewed through the SageMaker console of your AWS account.

In [ ]:
predictor.delete_endpoint()

# Reference
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)